<h1>药物</h1>

In [1]:
with open('Davis.txt', 'r') as f:
    lines = f.readlines()

data = []
for line in lines:
    parts = line.strip().split(' ', 4)
    if len(parts) == 5:
        compound_id, protein_name, smiles, rest = parts[0], parts[1], parts[2], parts[3] + ' ' + parts[4]
        sequence, label = rest.rsplit(' ', 1)
        data.append({
            'compound_id': compound_id,
            'protein_name': protein_name,
            'smiles': smiles,
            'sequence': sequence,
            'label': int(label)
        })

In [2]:
print(len(data))

25772


In [3]:
data

[{'compound_id': '11314340',
  'protein_name': 'AAK1',
  'smiles': 'CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC=C4)N',
  'sequence': 'MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAEGGFAIVFLVRTSNGMKCALKRMFVNNEHDLQVCKREIQIMRDLSGHKNIVGYIDSSINNVSSGDVWEVLILMDFCRGGQVVNLMNQRLQTGFTENEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLHDRGHYVLCDFGSATNKFQNPQTEGVNAVEDEIKKYTTLSYRAPEMVNLYSGKIITTKADIWALGCLLYKLCYFTLPFGESQVAICDGNFTIPDNSRYSQDMHCLIRYMLEPDPDKRPDIYQVSYFSFKLLKKECPIPNVQNSPIPAKLPEPVKASEAAAKKTQPKARLTDPIPTTETSIAPRQRPKAGQTQPNPGILPIQPALTPRKRATVQPPPQAAGSSNQPGLLASVPQPKPQAPPSQPLPQTQAKQPQAPPTPQQTPSTQAQGLPAQAQATPQHQQQLFLKQQQQQQQPPPAQQQPAGTFYQQQQAQTQQFQAVHPATQKPAIAQFPVVSQGGSQQQLMQNFYQQQQQQQQQQQQQQLATALHQQQLMTQQAALQQKPTMAAGQQPQPQPAAAPQPAPAQEPAIQAPVRQQPKVQTTPPPAVQGQKVGSLTPPSSPKTQRAGHRRILSDVTHSAVFGVPASKSTQLLQAAAAEASLNKSKSATTTPSGSPRTSQQNVYNPSEGSTWNPFDDDNFSKLTAEELLNKDFAKLGEGKHPEKLGGSAESLIPGFQSTQGDAFATTSFSAGTAEKRKGGQTVDSGLPLLSVSDPFIPLQVPDAPEKLIEGLKSPDTSLLLPDLLPMTDPFGSTSDAVIEKADVAVESLIPGLEPPVPQRLPSQTESVTSNRTDSLTGED

In [4]:
smiles = set()
for smile in data:
    smiles.add(smile['smiles'])

In [5]:
for smile in smiles:
    print(smile)

CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC=C4)N
CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=NC(=NC(=C3)N4CCN(CC4)CCO)C
CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C)NC4=NC(=CS4)C5=CN=CC=C5
CNC(=O)C1=CC=CC=C1SC2=CC3=C(C=C2)C(=NN3)C=CC4=CC=CC=N4
COC1=CC=C(C=C1)COC2=C(C=C(C=C2)CC3=CN=C(N=C3N)N)OC
CN1CCC(C(C1)O)C2=C(C=C(C3=C2OC(=CC3=O)C4=CC=CC=C4Cl)O)O
CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C)NC4=NC=CC(=N4)C5=CN=CC=C5
C1COCCN1C2=NC(=NC3=C2OC4=C3C=CC=N4)C5=CC(=CC=C5)O
CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC(=C(C=C3)Cl)C(F)(F)F
CC1=CC=C(C=C1)N2C(=CC(=N2)C(C)(C)C)NC(=O)NC3=CC=C(C4=CC=CC=C43)OCCN5CCOCC5
CS(=O)(=O)CCNCC1=CC=C(O1)C2=CC3=C(C=C2)N=CN=C3NC4=CC(=C(C=C4)OCC5=CC(=CC=C5)F)Cl
CC12C(CC(O1)N3C4=CC=CC=C4C5=C6C(=C7C8=CC=CC=C8N2C7=C53)CNC6=O)(CO)O
CC1=C(C=C(C=C1)C(=O)NC2=CC(=CC(=C2)N3C=C(N=C3)C)C(F)(F)F)NC4=NC=CC(=N4)C5=CN=CC=C5
C1=CC=C2C(=C1)C(=NN=C2NC3=CC=C(C=C3)Cl)CC4=CC=NC=C4
C1CC1CONC(=O)C2=C(C(=C(C=C2)F)F)NC3=C(C=C(C=C3)I)Cl
CN1C=C(C2=CC=CC=C21)C3=C(C(=O)NC3=O)C4=CN(C5=CC=

In [6]:
smiles = list(smiles)

In [7]:
print(len(smiles))

68


In [8]:
import torch
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cuda')

In [9]:
# from transformers import AutoTokenizer, RobertaModel
# 
#  = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-10M-MTR")
#  = RobertaModel.from_pretrained("DeepChem/ChemBERTa-10M-MTR").to(device)
# model.eval()

from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
model = AutoModel.from_pretrained("DeepChem/ChemBERTa-77M-MLM").to(device)
model.eval()

Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(600, 384, padding_idx=1)
    (position_embeddings): Embedding(515, 384, padding_idx=1)
    (token_type_embeddings): Embedding(1, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.144, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-2): 3 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.109, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dr

tokenizer里包括里input_ids和attention_mask

In [10]:
input = tokenizer(smiles,padding=True, truncation=True,max_length=512, return_tensors="pt").to(device)

In [11]:
input

{'input_ids': tensor([[12, 16, 16,  ...,  0,  0,  0],
        [12, 16, 16,  ...,  0,  0,  0],
        [12, 16, 16,  ...,  0,  0,  0],
        ...,
        [12, 16, 20,  ...,  0,  0,  0],
        [12, 16, 23,  ...,  0,  0,  0],
        [12, 16, 16,  ...,  0,  0,  0]], device='mps:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='mps:0')}

In [12]:
with torch.no_grad():
    output = model(**input)

In [13]:
output.pooler_output

tensor([[ 0.1096,  0.0373,  0.0439,  ...,  0.0718,  0.1327,  0.0790],
        [ 0.0281,  0.0995,  0.0383,  ...,  0.1635,  0.1342,  0.0254],
        [ 0.0729,  0.0937,  0.0226,  ...,  0.1251,  0.1108,  0.0623],
        ...,
        [-0.1298,  0.1838, -0.0087,  ...,  0.1975,  0.0716,  0.0430],
        [ 0.0905,  0.0472, -0.0488,  ...,  0.1447,  0.2156,  0.0486],
        [ 0.0575,  0.0628, -0.0586,  ...,  0.1161,  0.0929,  0.0158]],
       device='mps:0')

In [14]:
(output.pooler_output).shape

torch.Size([68, 384])

In [15]:
output.last_hidden_state.shape

torch.Size([68, 94, 384])

In [16]:
output.last_hidden_state.mean(dim=1)

tensor([[ 0.1172,  0.0739, -0.0100,  ...,  0.1643,  0.0652,  0.0051],
        [ 0.0571,  0.1526, -0.2340,  ...,  0.1003, -0.0155,  0.0106],
        [ 0.1075,  0.0827, -0.1571,  ...,  0.1955,  0.0673, -0.0724],
        ...,
        [ 0.1360,  0.0359, -0.0423,  ...,  0.2198, -0.0298, -0.0771],
        [-0.0088,  0.0455, -0.1814,  ..., -0.0111, -0.0426,  0.0247],
        [ 0.0993,  0.0128, -0.1294,  ...,  0.1383,  0.0832,  0.0283]],
       device='mps:0')

In [17]:
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

这个要保存为cpu不然保存不了，gpu的话怕适配不了所以要先保存为cpu

In [18]:
features = output.last_hidden_state.cpu()
torch.save(features, 'davis_ligands.pt')

In [19]:
import torch

features = torch.load('davis_ligands.pt')
print(features.shape)

torch.Size([68, 94, 384])


/var/folders/43/_y7ymkkx4hq3tfrsgr2r2hbw0000gn/T/ipykernel_55291/3802212158.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  features = torch.load('davis_ligands.pt')


<h1>蛋白质</h1>

In [20]:
with open('Davis.txt', 'r') as f:
    lines = f.readlines()

data = []
for line in lines:
    parts = line.strip().split(' ', 4)
    if len(parts) == 5:
        compound_id, protein_name, smiles, rest = parts[0], parts[1], parts[2], parts[3] + ' ' + parts[4]
        sequence, label = rest.rsplit(' ', 1)
        data.append({
            'compound_id': compound_id,
            'protein_name': protein_name,
            'smiles': smiles,
            'sequence': sequence,
            'label': int(label)
        })

proteins = set()
for protein in data:
    proteins.add(protein['sequence'])

In [21]:
proteins =list(proteins)
for protein in proteins:
    print(protein)

...


In [22]:
import torch
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cuda')
import re

# Load model directly
from transformers import T5Tokenizer, T5EncoderModel

local_path_model = "prot_t5_xl_uniref50/"
tokenizer = T5Tokenizer.from_pretrained(local_path_model)
model = T5EncoderModel.from_pretrained(local_path_model).to(device)
model.eval()

sequence_examples = proteins
# this will replace all rare/ambiguous amino acids by X and introduce white-space between all amino acids
sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in sequence_examples]

# tokenize sequences and pad up to the longest sequence in the batch
ids = tokenizer.batch_encode_plus(sequence_examples, add_special_tokens=True, padding="longest")
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

# generate embeddings
with torch.no_grad():
    embedding_repr = model(input_ids=input_ids,attention_mask=attention_mask)

# extract embeddings for the first ([0,:]) sequence in the batch while removing padded & special tokens ([0,:7]) 
emb_0 = embedding_repr.last_hidden_state[0,:7] # shape (7 x 1024)
print(f"Shape of per-residue embedding of first sequences: {emb_0.shape}")
# do the same for the second ([1,:]) sequence in the batch while taking into account different sequence lengths ([1,:8])
emb_1 = embedding_repr.last_hidden_state[1,:8] # shape (8 x 1024)

# if you want to derive a single representation (per-protein embedding) for the whole protein
emb_0_per_protein = emb_0.mean(dim=0) # shape (1024)

print(f"Shape of per-protein embedding of first sequences: {emb_0_per_protein.shape}")

OSError: Incorrect path_or_model_id: 'prot_t5_xl_uniref50/'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:

with open('Davis.txt', 'r') as f:
    lines = f.readlines()

data = []
for line in lines:
    parts = line.strip().split(' ', 4)
    if len(parts) == 5:
        compound_id, protein_name, smiles, rest = parts[0], parts[1], parts[2], parts[3] + ' ' + parts[4]
        sequence, label = rest.rsplit(' ', 1)
        data.append({
            'compound_id': compound_id,
            'protein_name': protein_name,
            'smiles': smiles,
            'sequence': sequence,
            'label': int(label)
        })

proteins = set()
for protein in data:
    proteins.add(protein['sequence'])

proteins = list(proteins)
print(len(proteins))

In [ ]:
from transformers import AutoTokenizer, AutoModel
#local_path = "/Volumes/PASSPORT/FinalYearProject/ChemBERTa-77M-MLM"
tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
model = AutoModel.from_pretrained("DeepChem/ChemBERTa-77M-MLM").to(device)
model.eval()

｜分子相识性

In [ ]:
from torch.nn.functional import cosine_similarity
import torch
device = torch.device("mps")

from transformers import AutoTokenizer, AutoModel
local_path = "/Volumes/PASSPORT/FinalYearProject/ChemBERTa-77M-MLM"
tokenizer = AutoTokenizer.from_pretrained(local_path)
model = AutoModel.from_pretrained(local_path).to(device)

smiles_1 = ["CCO"]
smiles_2 = ["CCN"]

tokens_1 = tokenizer(smiles_1, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
tokens_2 = tokenizer(smiles_2, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)

with torch.no_grad():
    emb_1 = model(**tokens_1).pooler_output  # 形状: (1, 768)
    emb_2 = model(**tokens_2).pooler_output

similarity = cosine_similarity(emb_1, emb_2)
print(f"SMILES 相似度: {similarity.item()}")
